In [25]:

%matplotlib inline
import matplotlib.pyplot as mp
import urllib2
import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

#variables
Basic_Stats = pd.read_csv("./nflstatistics/Basic_Stats.csv");
experience_level = Basic_Stats['Experience'];
standardized_experiences = [];
for val in experience_level:
    #turn data into standardized form
    #IE: Rookie and 1 Season should end up as the same value
    if "Rookie" in str(val):
       standardized_experiences.append
    elif "1" in val:
       standardized_experiences.append
    elif "3" in val:
       standardized_experiences.append
print standardized_experiences;

  

TypeError: argument of type 'float' is not iterable